In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
from IPython.display import Image, display

import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [2]:
from typing import Literal
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

class SupervisorDecision(BaseModel):
    next: Literal["researcher", "coder", "FINISH"]

model = llm
model = model.with_structured_output(SupervisorDecision)

agents = ["researcher", "coder"]

system_prompt_part_1 = f"""Anda adalah pengawas yang ditugaskan mengelola
percakapan antara pekerja berikut: {agents}. Mengingat permintaan pengguna berikut,
tanggapi dengan pekerja yang akan bertindak berikutnya. Setiap pekerja akan melakukan
tugas dan menanggapi dengan hasil dan status mereka. Ketika selesai,
tanggapi dengan FINISH."""

system_prompt_part_2 = f"""Mengingat percakapan di atas, siapa yang harus bertindak berikutnya? Atau
haruskah kita FINISH? Pilih salah satu: {', '.join(agents)}, FINISH"""

def supervisor(state):
    messages = [
        ("system", system_prompt_part_1),
        *state["messages"],
        ("system", 	system_prompt_part_2)
    ]
    return model.invoke(messages)